In [ ]:
from extended_source_files import *
from plot_helpers import *

from raster_compare.plots import PlotBase

import math

In [ ]:
mask_to_snow_depth()

# Snow Depth Difference

In [ ]:
COLORMAP = PlotColor.mpl_colormap

CASI_COLORS = ['dodgerblue', 'forestgreen', 'brown', 'orange']

AREA_PLOT_OPTS = dict(
    nrows=1, ncols=2, sharex=True, sharey=True, figsize=(16,12)
)

def area_plot():
    fig, axes = plt.subplots(**AREA_PLOT_OPTS)

    for ax in axes:
        ax.tick_params(axis='both', direction='inout', length=7)
        ax.xaxis.set_ticks_position('both')
        ax.yaxis.set_ticks_position('both')
        
        ax.set_facecolor('whitesmoke')
        ax.imshow(
            hillshade_snow_on,
            extent=sfm_snow_depth.extent,
            cmap='gray', clim=(1, 255), alpha=0.3,
        )
    return fig, axes

def insert_sd_colorbar(ax, data):
    PlotBase.insert_colorbar(
        ax, data, 
        'Snow Depth Difference (m)', 
        right=0.90, rect=[0.91, 0.22, 0.022, 0.608]
    )

In [ ]:
np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    sd_difference_values,
    copy=False
)
plot_data = sd_difference_values[np.isfinite(sd_difference_values)]

classification_plot = np.ma.masked_where(
    np.isfinite(sd_difference_values).mask,
    casi_classification,
).astype(np.int8);

In [ ]:
HIST_BIN_WIDTH = .10
bins = np.concatenate((
    [math.floor(plot_data.min())],
    np.arange(-2., 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(plot_data.max())],
))
imshow_opts = dict(
    extent=sfm_snow_depth.extent,
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=COLORMAP.N,
    ),
    cmap=COLORMAP,
)

In [ ]:
fig, (ax1, ax2) = area_plot()
fig.subplots_adjust(wspace=.15)

im_data = ax1.imshow(
    sd_difference_values,     
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")
PlotBase.insert_colorbar(ax1, im_data, 'Snow Depth Difference (m)')

im_data = ax2.imshow(
    classification_plot, 
    extent=sfm_snow_depth.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax2.set_title("Snow Depth Differences - Classification")
PlotBase.insert_colorbar(ax2, im_data, 'Classification');

In [ ]:
high_elevation = np.ma.masked_where(
    dem_values <= 3500,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    dem_values > 3500,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()

ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - High Elevation >= 3500m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - Low Elevation < 3500")

insert_sd_colorbar(ax2, im_data);

In [ ]:
high_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3100, 3200).mask,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3800, 3900).mask,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()
ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - 3100m < Elevation < 3200m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - 3800 < Elevation < 3900")

insert_sd_colorbar(ax2, im_data);

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sd_difference_values, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")

im_data = ax2.imshow(
    sfm_snow_free_values - dem_values, 
    **imshow_opts,
)
ax2.set_title("Snow Free - Reference DEM")

insert_sd_colorbar(ax2, im_data);

In [ ]:
data = [
    {
        'data': sfm_snow_free_values - dem_values,
        'label': 'SfM snow free - DEM',
        'color': 'dodgerblue',
    },
]

ax = plot_histogram(data, (-6, 6), figsize=(10, 8))
ax.set_title('Elevation Differences (SFM snow free - Reference DEM)');